**Basic metadata pie charts**

In [1]:
# load data
all_data = readRDS('../processed_data/single_cell/stats_input_data_0825.rds')
f = readRDS('../processed_data/single_cell/pseudobulk_degs_0825.rds')
df = all_data$summary
df$AD = ifelse(df$niareagansc<3, 'AD', 'non-AD')

In [114]:
# Install and load necessary libraries
# install.packages("ggplot2")
# install.packages("patchwork")
# install.packages("dplyr")
# install.packages("ggsci")
library(ggplot2)
library(patchwork)
library(dplyr)
library(ggsci)
library(cowplot)

# Create a function to generate a pie chart for a specific column
create_pie_chart <- function(df, column_name) {
  df[[column_name]] <- as.character(df[[column_name]])
  
  percentages <- df %>%
    group_by(!!sym(column_name)) %>%
    summarise(count = n()) %>%
    mutate(percentage = count / sum(count) * 100)
  
  ggplot(percentages, aes(x = "", y = percentage, fill = !!sym(column_name))) +
    geom_bar(width = 1, stat = "identity") +
    coord_polar("y", start = 0) +
    theme_void() +
    scale_fill_npg(guide = guide_legend(title.position = "top")) +  # Use the NPG color palette
    ggtitle(column_name) +
    geom_text(aes(label = paste0(round(percentage, 1), "%")), 
              position = position_stack(vjust = 0.5), 
              color = "white", size = 5)     # Center the legend title
}

# Create individual pie charts
column_names = c('AD', 'ceradsc', 'braaksc', 'msex')

pie_charts1 <- lapply(column_names, function(col) create_pie_chart(df, col))

#pie_charts1 <- lapply(column_names, function(col) create_pie_chart(df[df$LOF==0,], col))
#pie_charts2 <- lapply(column_names, function(col) create_pie_chart(df[df$LOF==1,], col))

# Arrange the pie charts in a 2x4 grid
pie_grid <- wrap_plots(c(pie_charts1), ncol = 4)

pdf('../pdf_figures/pie_chart.pdf')
pie_grid      
dev.off()                      
# # Wrap each pie chart with a grey box
# pie_charts_with_boxes <- lapply(pie_charts, function(p) {
#   ggdraw(p) + 
#     theme(plot.background = element_rect(color = "grey", fill = NA, size = 0.5))
# })

# # Arrange the pie charts in a 2x4 grid
# pie_grid <- wrap_plots(pie_charts_with_boxes, ncol = 4) + plot_layout(guides = "collect")
                      
# pie_grid

png 
  2